<a href="https://colab.research.google.com/github/VivianMonzon/FAL_prediction/blob/main/Colab/ML_FA_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install dependencies/clone repo
!pip install biopython &> /dev/null
!git clone https://github.com/VivianMonzon/FAL_prediction --quiet

In [ ]:
#@title Installing miniconda
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-py37_4.10.3-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT &> /dev/null
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX &> /dev/null

In [ ]:
#@title Install hmmer and muscle
!conda install --channel defaults conda python=3.7 --yes &> /dev/null
!conda update --channel defaults --all --yes &> /dev/null

import sys
_ = (sys.path
        .append("/usr/local/lib/python3.7/site-packages"))
!conda install -c bioconda hmmer --yes &> /dev/null
!wget https://www.drive5.com/muscle/downloads3.8.31/muscle3.8.31_i86linux64.tar.gz &> /dev/null
!tar -zxvf muscle3.8.31_i86linux64.tar.gz &> /dev/null

In [ ]:
#@title Input protein sequence or sequence fasta file name (+path if not in current directory), then hit `Runtime` -> `Run all`

query_sequence = 'MFFKRQKGQYHEVERVTRFKLIKSGKHWLRAATSQFGLLRLMKGADISSVEVKVAEEQSV EKGGLNYLKGIIATGAVLGGAVVTSSSVYAEEEQALEKVIDTRDVLATRGEAVLSEEAAT TLSSEGANPVESLSDTLSASESASANSVSTSISISESFSVSASASLSSSSSLSQSSSESA SASESLSVSASTSQSFSSTTSSTQSSNNESLISSDSSNSLNTNQSVSARNQNARVRTRRA VAANDTEAPQVKSGDYVVYRGESFEYYAEITDNSGQVNRVVIRNVEGGANSTYLSPNWVK YSTENLGRPGNATVQNPLRTRIFGEVPLNEIVNEKSYYTRYIVAWDPSGNATQMVDNANR NGLERFVLTVKSQNEKYDPADPSVTYVNNLSNLSTSEREAVAAAVRAANPNIPPTAKITV SQNGTVTITYPDKSTDTIPANRVVKDLQISKSNSASQSSSVSASQSASTSVSASISASMS ASVSVSTSASTSASVSASESASTSASVSASESASTSASVSASKSSSTSASVSASESASTS ASVSASESASTSASVSASESASTSASVSASTSASTSASVSASESASTSASVSASESASTS ASVSASESASTSASVSASESASTSASVSASESSSTSASVSASESASTSASVSASESASTS ASVSASTSASTSASVSASTSASTSASVSASTSASTSASVSASESASTSASVSASESASTS ASVSASTSASTSASVSASTSASTSASVSASESASTSASVSASTSASTSASVSASESASTS ASVSASTSASTSASVSASESASTSASVSASESASTSASVSASTSASTSASVSASESASTS ASVSASESASTSASVSASESASTSASVSASTSASTSASVSASESASTSASVSASESASTS ASVSASESASTSASVSASESASTSASVSASTSASTSASVSASESASTSASVSASESASTS ASVSASESASTSASVSASESASTSASISASESASTSASVSASESASTSASVSASTSASTS ASVSASESASTSASVSASESASTSASVSASESASTSASVSASESASTSASVSASESASTS ASVSASTSASTSASVSASESSSTSASVSASESASTSSSVSASESASTSASVSASESASTS ASVSASESASTSASVSASESASTSASVSASESASTSASVSASESASTSASVSASESASTS ASVSASTSASTSASVSASESASTSASVSASESASTSASVSASTSASTSASVSASESASTS ASVSASESASTSASVSASESASTSASVSASTSASTSASVSASESASTSASVSASESASMS ASVSASESASTSASVSASESASTSASVSASESASTSASVSASESASTSASVSASESASTS ASVSASESASTSASVSASESAYTSASASASESASTSASISASESASTSASVSASESAYTS ASVSASESGSTSASVSASESASTSASVSASESASTSASVSASTSASTSASVSASESSSTS ASVSASESASTSASVSASESASTSASVSASTSASTSASVSASESASTSASVSASESASTS ASVSASESASTSASVSTSESASTSASVSASESASTSASVSASESASTSASVSASESSSTS ASVSASESASTSASVSASESASTSASVSASESASTSASVSASESASTSVSVSASESASTS ASVSASESASSSASVSASKSASMSASVLASESASTSASVSASESASTSASVSASESASTS ASVSASESASTSASVSASESASTSASVSASESASTSASVSASESASTSASVSASTSASTS ASVSASESASTSASVSASESASTSASVSASESASTSASVSASESVSANESASTSASVSAS TSASTSASVSSSESASTSASVSASESASTSASVSASESASTSASVSASESASISASISASESSSTSASVSASESASTSASVSASTSTSTSASVSASESASTSASVFASESASTSASVSASESASTSASVSASTSASTSASVSASESASTSASISASESASTSASISASESSSTSASVSASTSASTSASVSASESTSTSVSISASESVSISTSVSQSMSVSESLSLSVSTSTLHSQLNGIYESELNSLSLSESLSMSQSLSQSLSDSQSTSATQSMHDRISKGQLPRTGESESKASILALGIGALGLAFKKRKKNESED' #@param {type:"string"}
sequence_file = '' #@param {type:"string"}

In [ ]:
#@title Check sequence/sequence file & create folders (& seq file)
from Bio import SeqIO
from FAL_prediction.Colab import feature

def check_fasta_file(filename):
  with open(filename, "r") as handle:
        fasta = SeqIO.parse(handle, "fasta")
        return any(fasta)

if query_sequence:
  if not sequence_file:
    query_sequence = query_sequence.replace(" ", "")
    feature.query_seq_to_file(query_sequence)
    fh_fasta = 'query_seq.fa'
    print('Sequence file from query seq. created')
  else:
    raise ValueError('Please use either query sequence or give a sequence file name!')
if sequence_file:
  if not query_sequence:
    if check_fasta_file(sequence_file) is True:
      print('Sequence file is {}'.format(sequence_file))
      fh_fasta = sequence_file
    if check_fasta_file(sequence_file) is False:
      print('Please check the fasta file')
  else:
    raise ValueError('Please use either query sequence or give a sequence file name!')

feature.create_folder('analysis', 'results')

Sequence file from query seq. created
analysis folder created
results folder created


In [ ]:
#@title Run hmmersearch 
!hmmsearch --cut_ga --domtblout analysis/input_seq_adh_dom.tbl FAL_prediction/database/adh_dom_hmms.hmm $fh_fasta > analysis/input_seq_adh_dom.out
!hmmsearch --cut_ga --domtblout analysis/input_seq_stalk_dom.tbl FAL_prediction/database/stalk_dom_hmms.hmm $fh_fasta > analysis/input_seq_stalk_dom.out
!hmmsearch --cut_ga --domtblout analysis/input_seq_anchor_dom.tbl FAL_prediction/database/anchor_dom_hmms.hmm $fh_fasta > analysis/input_seq_anchor_dom.out


Error: GA bit thresholds unavailable on model GspA_SrpA_N



In [ ]:
#@title Run IUPRED
!python3.7 FAL_prediction/Colab/scripts/iupred2a.py $fh_fasta "long" > analysis/input_seq_iupred.tab

In [ ]:
#@title Run T-REKS
!java -Xmx4G -jar FAL_prediction/Colab/scripts/T-ReksHPC.jar -f $fh_fasta -t analysis/input_seq_treks.tsv -a analysis/input_seq_treks.aln -m /content/muscle3.8.31_i86linux64 -s 0.7 -S 5 -L 50 > analysis/input_seq_treks.log

In [ ]:
#@title Collect features
from Bio.SeqIO.FastaIO import SimpleFastaParser
import pandas as pd
feature.collect_features.adapt_hmmsearch('analysis/input_seq_adh_dom.tbl', 'analysis/input_seq_adh_dom_adapt.tbl')
feature.collect_features.adapt_hmmsearch('analysis/input_seq_stalk_dom.tbl', 'analysis/input_seq_stalk_dom_adapt.tbl')
feature.collect_features.adapt_hmmsearch('analysis/input_seq_anchor_dom.tbl', 'analysis/input_seq_anchor_dom_adapt.tbl')
df_combined = pd.DataFrame(columns=['ID', 'length', 'Hydro_portion', 'Charge_portion', 
                                    'rel_entropy', 'R', 'H', 'K', 'D', 'E', 'S', 'T', 
                                    'N', 'Q', 'C', 'G', 'P', 'A', 'V', 'I', 'L', 'M', 
                                    'F', 'Y', 'W'])
fh_seq_file = open(fh_fasta)
for name, seq in SimpleFastaParser(fh_seq_file):
  if '|' in name:
    name = name.split('|')[1].split('|')[0]
  if ' ' in name:
    name = name.split(' ')[0]
  if '.' in name:
    name = name.split('.')[0]
  df_hydro_charge = feature.collect_features.hydro_charge(name, seq)
  df_aa_comp = feature.collect_features.aa_comp(name, seq)
  df_len = pd.DataFrame({'ID': [name], 'length': [len(seq)]})
  df_merged = df_len.merge(df_hydro_charge, how='left').merge(df_aa_comp, how='left')
  df_combined = df_combined.append(df_merged)
fh_seq_file.close()
df_iupred = feature.collect_features.adapt_iupred('analysis/input_seq_iupred.tab')
df_treks = feature.collect_features.treks('analysis/input_seq_treks.tsv')
df_anchor = feature.collect_features.anchor_search(fh_fasta, 'analysis/input_seq_anchor_dom_adapt.tbl')
df_adh = feature.collect_features.any_adh('analysis/input_seq_adh_dom_adapt.tbl')
df_stalk = feature.collect_features.number_stalk('analysis/input_seq_stalk_dom_adapt.tbl')
df_input_seq = df_combined.merge(df_iupred, how='left').merge(df_treks, how='left').merge(
    df_anchor, how='left').merge(df_adh, how='left').merge(df_stalk, how='left').fillna(0)
df_input_seq = df_input_seq[['ID', 'length', 'treks_07', 'Any_anchor', 'Stalks', 'Any_adh', 
                             'frac_disordered', 'Hydro_portion', 'Charge_portion', 'rel_entropy', 'R', 'H', 
                             'K', 'D', 'E', 'S', 'T', 'N', 'Q', 'C', 'G', 'P', 'A', 'V', 'I', 'L', 'M', 'F', 'Y', 'W']]

df_input_seq.to_csv('results/input_seq_feature.csv', index=False)

In [ ]:
#@title RF prediction
from FAL_prediction.Colab import RF_prediction
RF_prediction.ML_prediction.run_random_forest('results/input_seq_feature.csv', 
                                              'FAL_prediction/database/training_data.csv', 
                                              'results', 'input_seq_results')
print('Your results are saved under results/input_seq_results.tsv')

Your results are saved under results/input_seq_results.tsv


In [ ]:
#@title Head of resuls file:
!head results/input_seq_results.tsv

ID	Prediction	Prediction_score	length	Hydro_portion	Charge_portion	rel_entropy	R	H	K	D	E	S	T	N	Q	C	G	P	A	V	I	L	M	F	Y	W
query_sequence	1	0.82	2178	0.33	0.1	1.58	1.24	0.18	1.24	0.83	6.8	42.33	8.95	1.74	1.19	0.0	1.38	0.69	19.93	8.17	1.42	2.02	0.41	0.51	0.83	0.14
